## the cake baking code example from 
https://www.youtube.com/watch?v=WGT9_cEImAk
<br>
JuliaCon 2019

In [1]:
abstract type State end
abstract type Transition end

InvalidState = ArgumentError

@enum Ingredients eggs flour sugar water butter baking_soda

In [2]:
struct NeedIngredients <: State
    gathered::Set{Ingredients}
    NeedIngredients() = new(Set())
end

struct Gather <: Transition
    ingredient::Ingredients
end

In [3]:
struct MixBatter <: State
    remaining::Array{Ingredients}
    MixBatter() = new([flour, sugar, baking_soda, water, eggs, butter])
end

struct Add <: Transition
    ingredient::Ingredients
end

In [4]:
struct RuinedBatter <: State end
struct RawBatter <: State end

struct Bake <: Transition
    temp::UInt64
end

struct TastyCake <: State end
struct BurntCake <: State end

## stop and think here
"these are states and transitions described as types. how do we implemen a state machine?"

In [5]:
step(::State, ::Transition) = throw(InvalidState)
# by default, any transition from any state is invalid. 
# next we are going to define what are the valid transitions

step (generic function with 1 method)

In [6]:
function step(state::NeedIngredients, transition::Gather)
    push!(state.gathered, transition.ingredient)
    state.gathered != Set(instances(Ingredients)) ? state : MixBatter()
end

step (generic function with 2 methods)

In [7]:
function step(state::MixBatter, transition::Add)
    if state.remaining[1] == transition.ingredient
        popfirst!(state.remaining)
        isempty(state.remaining) ? RawBatter() : state
    else
        RuinedBatter()
    end
end

step (generic function with 3 methods)

In [8]:
function step(::RawBatter, transition::Bake) 
    transition.temp > 350 ? BurntCake() : TastyCake()
end

step (generic function with 4 methods)

In [9]:
step(NeedIngredients(), Gather(eggs))

NeedIngredients(Set(Ingredients[eggs]))

In [12]:
recipe = [
    Gather(flour)
    Gather(eggs)
    Gather(water)
    Gather(baking_soda)
    Gather(butter)
    Gather(sugar)
    Add(flour)
    Add(sugar)
    Add(baking_soda)
    Add(water)
    Add(eggs)
    Add(butter)
    Bake(622)
]

13-element Vector{Transition}:
 Gather(flour)
 Gather(eggs)
 Gather(water)
 Gather(baking_soda)
 Gather(butter)
 Gather(sugar)
 Add(flour)
 Add(sugar)
 Add(baking_soda)
 Add(water)
 Add(eggs)
 Add(butter)
 Bake(0x000000000000026e)

In [13]:
 reduce(step, recipe, init=NeedIngredients())

BurntCake()